In [1]:
import subprocess
import sys

def install_and_import(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_libraries = ["os", "pandas", "requests", "datetime"]

for lib in required_libraries:
    install_and_import(lib)

import os
import pandas as pd
import requests
from datetime import datetime

In [2]:
def download_vhi_data(region_id):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"vhi/vhi_region_{region_id}_{timestamp}.csv"
    
    if not os.path.exists("vhi"):
        os.makedirs("vhi")

    try:
        response = requests.get(url)
        response.raise_for_status()
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"Дані для області {region_id} завантажено: {filename}")
        return filename
    except requests.RequestException as e:
        print(f"Помилка при завантаженні даних для області {region_id}: {e}")
        return None

def download_all_vhi_data():
    latest_files = {}
    for region_id in range(1, 28):
        file = download_vhi_data(region_id)
        if file:
            latest_files[region_id] = file
    return latest_files

In [3]:
def load_vhi_data(filepath):
    if not filepath or not os.path.exists(filepath):
        print("Файл не знайдено.")
        return None
    try:
        df = pd.read_csv(filepath, index_col=False, header=1)
        df.columns = ["year", "week", "SMN", "SMT", "VCI", "TCI", "VHI"]
        df.dropna(inplace=True)
        df = df[df["VHI"] != -1] 
        return df
    except Exception as e:
        print(f"Помилка при зчитуванні файлу: {e}")
        return None


In [4]:
def rename_regions(df):
    region_mapping = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
                      11: 9, 13: 10, 14: 11, 15: 12, 16: 13, 17: 15, 18: 14, 19: 16,
                      21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    df.replace({"area": region_mapping}, inplace=True)
    return df

In [5]:
def analyze_vhi_data(df, region_id, year):
    df_filtered = df[(df["year"] == year)]
    if df_filtered.empty:
        print(f"Немає даних для {year} року.")
        return
    print(f"Аналіз VHI за {year} рік:")
    for region in df_filtered["area"].unique():
        region_data = df_filtered[df_filtered["area"] == region]
        print(f"Область {region}: Мін: {region_data['VHI'].min()}, Макс: {region_data['VHI'].max()}, \
              Середнє: {region_data['VHI'].mean():.2f}, Медіана: {region_data['VHI'].median():.2f}")


In [6]:
def detect_drought_years(df):
    drought_years = []
    for year in df["year"].unique():
        drought_regions = df[(df["year"] == year) & (df["VHI"] < 15)]
        if len(drought_regions) > 5: 
            drought_years.append(year)
    print("Роки екстремальних посух:", drought_years)


In [30]:
def get_vhi_series(df, year):
    data = df[df["year"] == year]
    print(f"VHI-ряд для {year} по всіх регіонах:")
    print(data[["week", "area", "VHI"]])

In [19]:
latest_files = download_all_vhi_data()
all_data = []
for region_id, filepath in latest_files.items():
    df = load_vhi_data(filepath)
    if df is not None:
        df["area"] = region_id
        all_data.append(df)

Дані для області 1 завантажено: vhi/vhi_region_1_20250313_080720.csv
Дані для області 2 завантажено: vhi/vhi_region_2_20250313_080721.csv
Дані для області 3 завантажено: vhi/vhi_region_3_20250313_080723.csv
Дані для області 4 завантажено: vhi/vhi_region_4_20250313_080724.csv
Дані для області 5 завантажено: vhi/vhi_region_5_20250313_080726.csv
Дані для області 6 завантажено: vhi/vhi_region_6_20250313_080727.csv
Дані для області 7 завантажено: vhi/vhi_region_7_20250313_080728.csv
Дані для області 8 завантажено: vhi/vhi_region_8_20250313_080730.csv
Дані для області 9 завантажено: vhi/vhi_region_9_20250313_080731.csv
Дані для області 10 завантажено: vhi/vhi_region_10_20250313_080732.csv
Дані для області 11 завантажено: vhi/vhi_region_11_20250313_080734.csv
Дані для області 12 завантажено: vhi/vhi_region_12_20250313_080735.csv
Дані для області 13 завантажено: vhi/vhi_region_13_20250313_080737.csv
Дані для області 14 завантажено: vhi/vhi_region_14_20250313_080738.csv
Дані для області 15 зава

In [31]:
    analyze_vhi_data(full_df, None, "1991")
    detect_drought_years(full_df)
    get_vhi_series(full_df, "1992")

Аналіз VHI за 1991 рік:
Область 1: Мін: 27.18, Макс: 64.56,               Середнє: 46.54, Медіана: 45.03
Область 2: Мін: 31.92, Макс: 57.49,               Середнє: 44.08, Медіана: 42.69
Область 3: Мін: 28.76, Макс: 65.39,               Середнє: 46.96, Медіана: 47.45
Область 4: Мін: 32.67, Макс: 63.98,               Середнє: 45.73, Медіана: 40.55
Область 5: Мін: 36.04, Макс: 66.29,               Середнє: 47.21, Медіана: 46.18
Область 6: Мін: 33.48, Макс: 65.8,               Середнє: 45.94, Медіана: 41.35
Область 7: Мін: 31.77, Макс: 58.3,               Середнє: 45.13, Медіана: 44.53
Область 8: Мін: 40.28, Макс: 66.1,               Середнє: 50.79, Медіана: 46.88
Область 9: Мін: 30.12, Макс: 63.15,               Середнє: 42.14, Медіана: 40.68
Область 10: Мін: 27.59, Макс: 60.56,               Середнє: 45.46, Медіана: 46.19
Область 11: Мін: 26.31, Макс: 61.55,               Середнє: 45.69, Медіана: 46.05
Область 12: Мін: 33.14, Макс: 58.16,               Середнє: 43.66, Медіана: 43.14
Обла